## Prompt Engineering using ChatPromptTemplate and different sequential chains

- Start by importing necessary libraries
- read the openai api keys
- Define LLM model using Chat GPT
- Format prompts using ChatPromptTemplate
- Define LLM Chain for inference
- Examples using different types of prompt chains
  - SimpleSequentialChain
  - SequentialChain
  - MultiPromptChain

In [2]:
import os
import openai
from dotenv import load_dotenv, find_dotenv

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
_ = load_dotenv(find_dotenv())

In [6]:
import datetime

def get_llm():
    current_dt = datetime.datetime.now()
    target_dt = datetime.datetime(2024, 6, 12)
    if current_dt>target_dt:
        llm_model = "gpt-3.5-turbo"
    else:
        llm_model = "gpt-3.5-turbo-0301"
    return llm_model

In [7]:
import pandas as pd

data = pd.read_csv("Data.csv")

data.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Product  7 non-null      object
 1   Review   7 non-null      object
dtypes: object(2)
memory usage: 244.0+ bytes


In [9]:
data

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [11]:
llm_model_nm = get_llm()
print(llm_model_nm)

gpt-3.5-turbo


In [12]:
llm = ChatOpenAI(
    model=llm_model_nm,
    temperature=0.2
)

In [13]:
chat_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe the company\
    that makes product {product}?\
")

In [15]:
chat_prompt.messages[0].input_variables

['product']

## LLMChain with single input and single output

In [17]:
chain = LLMChain(llm=llm, prompt=chat_prompt)
product = "Weight management ACV Capsules"
chain.run(product)

'"TrimLife ACV Capsules"'

In [31]:
s = data.loc[[0],['Product']]

In [37]:
for i in s['Product']:
    product = i

In [38]:
print(product)

Queen Size Sheet Set


In [39]:
chain = LLMChain(llm=llm, prompt=chat_prompt)
chain.run(product)

'Regal Linens Co.'

## SimpleSequentialChain with single input and single output

In [40]:
from langchain.chains import SimpleSequentialChain

In [44]:
first_prompt = ChatPromptTemplate.from_template("\
Give the best company name to describe the given product:{product}.")

second_prompt = ChatPromptTemplate.from_template("\
Write a summary of 5 lines to best describe the company:{company_name}")

first_chain = LLMChain(llm=llm, prompt=first_prompt, output_key="company_name")
second_chain = LLMChain(llm=llm, prompt=second_prompt)

In [45]:
complete_chain = SimpleSequentialChain(chains=[first_chain, second_chain], verbose=True)

In [46]:
complete_chain.run(product)



> Entering new SimpleSequentialChain chain...
Regal Linens
Regal Linens is a luxury linen company that specializes in creating high-quality bedding and home textiles. They offer a wide range of products including sheets, duvet covers, and towels made from premium materials. With a focus on elegance and comfort, Regal Linens provides customers with luxurious options for their home decor needs. Their products are known for their durability and softness, making them a popular choice among those seeking a touch of sophistication in their living spaces. Regal Linens is committed to providing exceptional customer service and ensuring customer satisfaction with every purchase.

> Finished chain.


'Regal Linens is a luxury linen company that specializes in creating high-quality bedding and home textiles. They offer a wide range of products including sheets, duvet covers, and towels made from premium materials. With a focus on elegance and comfort, Regal Linens provides customers with luxurious options for their home decor needs. Their products are known for their durability and softness, making them a popular choice among those seeking a touch of sophistication in their living spaces. Regal Linens is committed to providing exceptional customer service and ensuring customer satisfaction with every purchase.'

## SequentialChain with multiple inputs

In [48]:
from langchain.chains import SequentialChain

In [49]:
llm = ChatOpenAI(model=llm_model_nm, temperature=0.9)

first_prompt = ChatPromptTemplate.from_template(
    "Translate the product review given by the text into U.S. English language. Text:{review}"
)
first_chain = LLMChain(llm=llm, prompt=first_prompt, output_key="eng_review")

second_prompt = ChatPromptTemplate.from_template(
    "Summarize the given review into Pros and Cons. Review:{eng_review}"
)
second_chain = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

third_prompt = ChatPromptTemplate.from_template(
    "What is the language of the review text? Review:{review}"
)
third_chain = LLMChain(llm=llm, prompt=third_prompt, output_key="language")

fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow-up response for the summary provided in the given language. Summary:{summary}, language:{language}."
)
fourth_chain = LLMChain(llm=llm, prompt=fourth_prompt, output_key="response_in_orig_lang")


In [51]:
complete_chain = SequentialChain(chains=[first_chain, second_chain, third_chain, fourth_chain], 
                                 input_variables=['review'],
                                 output_variables=['eng_review','summary','language','response_in_orig_lang'],
                                 verbose=True)

review = data.Review[6]
complete_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': 'Está lu bonita calienta muy rápido, es muy funcional, solo falta ver cuánto dura, solo llevo 3 días en funcionamiento.',
 'eng_review': 'Translation: It is very pretty, heats up very quickly, it is very functional, I just need to see how long it lasts, I have only been using it for 3 days.',
 'summary': 'Pros:\n- Very pretty design\n- Heats up quickly\n- Very functional\n\nCons:\n- Durability is uncertain, as the reviewer has only been using it for 3 days',
 'language': 'Spanish',
 'response_in_orig_lang': 'Gracias por compartir tus comentarios sobre el producto. Es bueno saber que el diseño es bonito, que se calienta rápidamente y que es muy funcional. Sin embargo, es comprensible que la durabilidad sea una preocupación, especialmente después de solo 3 días de uso. Sería útil continuar probándolo para evaluar mejor su resistencia a largo plazo. ¡Esperamos que siga funcionando bien para ti en el futuro! ¡Gracias de nuevo por tu opinión!'}

## MultiPromptChain and RouterChain

In [52]:
physics_template = """\
You are a physics professor with an excellent academic record. You will be asked a question given by input.\
You have to answer as per your understanding about the topic. You are honest and you accept when you don't know the answer.\
Here is the question:
{input}
"""

maths_template = """\
You are a mathematician and are good at answering questions related to simple as well as complex mathematical operations.\
You break a mathematical equation into individual components and solve each component and combine the results for the overall answer.\
Here is the question:
{input}
"""

geo_template = """\
You are a very intelligent geography professor and are very good at answering questions related to geography, \
when you don't know an answer you reply that you don't know the answer.\
Here is the question:
{input}
"""

history_template = """\
You are a very knowledgable history professor and are very good at answering questions related to geography, \
when you don't know an answer you reply that you don't know the answer.\
Here is the question:
{input}
"""

In [53]:
prompt_messages = [
    {
        "name":"PhysicsBot",
        "description":"Good at answering physics questions.",
        "prompt_template":physics_template
    },
    {
        "name":"MathsBot",
        "description":"Good at answering maths questions.",
        "prompt_template":maths_template
    },
    {
        "name":"GeoBot",
        "description":"Good at answering geography questions.",
        "prompt_template":geo_template
    },
    {
        "name":"HistoryBot",
        "description":"Good at answering history questions.",
        "prompt_template":history_template
    }
]

In [54]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import RouterOutputParser, LLMRouterChain

In [55]:
llm = ChatOpenAI(model=llm_model_nm, temperature=0.0)

In [57]:
destination_chains={}
for prompt in prompt_messages:
    p_name = prompt['name']
    p_template = ChatPromptTemplate.from_template(template=prompt['prompt_template'])
    llm_chain = LLMChain(llm=llm, prompt=p_template)
    destination_chains[p_name]=llm_chain

destinations = [f"{p['name']}:{p['description']}" for p in prompt_messages]
print(destinations)
destinations_str = "\n".join(destinations)
print(destinations_str)
    

['PhysicsBot:Good at answering physics questions.', 'MathsBot:Good at answering maths questions.', 'GeoBot:Good at answering physics questions.', 'HistoryBot:Good at answering physics questions.']
PhysicsBot:Good at answering physics questions.
MathsBot:Good at answering maths questions.
GeoBot:Good at answering physics questions.
HistoryBot:Good at answering physics questions.


In [59]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [60]:
multi_prompt_router_template = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [62]:
from langchain.prompts import PromptTemplate
router_template = multi_prompt_router_template.format(destinations=destinations_str)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

In [63]:
router_chain = LLMRouterChain.from_llm(llm=llm, prompt=router_prompt)

In [64]:
chain = MultiPromptChain(router_chain=router_chain,
                        destination_chains=destination_chains,
                        default_chain=default_chain,
                        verbose=True)

In [65]:
chain.run("Which is the largest ocean in the world?")



> Entering new MultiPromptChain chain...
GeoBot: {'input': 'Which is the largest ocean in the world?'}
> Finished chain.


'The largest ocean in the world is the Pacific Ocean.'

In [66]:
chain.run("What is Newton's second law?")



> Entering new MultiPromptChain chain...
PhysicsBot: {'input': "What is Newton's second law?"}
> Finished chain.


"Newton's second law of motion states that the acceleration of an object is directly proportional to the net force acting on it and inversely proportional to its mass. In mathematical terms, this law can be expressed as F = ma, where F is the net force acting on the object, m is the mass of the object, and a is the acceleration of the object. This law essentially explains how the motion of an object changes when a force is applied to it."

In [67]:
chain.run("Tell me recipe for pinacolada")



> Entering new MultiPromptChain chain...
None: {'input': 'Tell me recipe for pinacolada'}
> Finished chain.


'Ingredients:\n- 2 oz white rum\n- 4 oz pineapple juice\n- 2 oz coconut cream\n- 1 cup crushed ice\n- Pineapple wedge and maraschino cherry for garnish\n\nInstructions:\n1. In a blender, combine the white rum, pineapple juice, coconut cream, and crushed ice.\n2. Blend until smooth and creamy.\n3. Pour the mixture into a glass.\n4. Garnish with a pineapple wedge and maraschino cherry.\n5. Serve and enjoy your refreshing piña colada!'

In [68]:
chain.run("What is (84+104)/21")



> Entering new MultiPromptChain chain...
MathsBot: {'input': 'What is (84+104)/21'}
> Finished chain.


'To solve this equation, we first need to add 84 and 104 together:\n\n84 + 104 = 188\n\nThen, we divide the sum by 21:\n\n188 / 21 = 8.952\n\nTherefore, (84+104)/21 = 8.952.'

In [69]:
chain.run("Check if this solution is correct: 21+21=42")



> Entering new MultiPromptChain chain...
MathsBot: {'input': 'Check if this solution is correct: 21+21=42'}
> Finished chain.


'To check if the solution is correct, we need to perform the addition operation on the left side of the equation:\n\n21 + 21 = 42\n\nTherefore, the solution provided is indeed correct.'